In [1]:
import urllib
import lxml
from lxml import etree
from lxml import html
import pandas as pd
from lxml.html.soupparser import fromstring

In [2]:
def get_page(url,hdr):
    page = urllib.request.Request(url,headers =hdr)
    response = urllib.request.urlopen(page)
    rawtext = response.read()
    return str(rawtext)

hdr1 = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

hdr2 = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
        'Accept-Encoding': 'none',
        'Accept-Language': 'en-US,en;q=0.8',
        'Connection': 'keep-alive'}

hdrs = [hdr1,hdr2]


In [3]:
def collect_asin():
    url = 'https://www.amazon.com/s?k=%s&s=%s'%('drills','salesrank')
    rawtext = get_page(url,hdr1)
    tree = html.fromstring(rawtext)
    search_rows = tree.xpath("//*[@class='s-result-list s-search-results sg-row']")[0]
    asins = []
    for i in range(10):
        if not ('AdHolder' in search_rows[i].attrib['class']) and search_rows[i].attrib['data-asin'] !='':
            asins.append(search_rows[i].attrib['data-asin'])
    return asins


def amazon_table(tree,key):
    table =  tree.xpath("//*[@id='%s']"%key)[0]
    table = pd.read_html(etree.tostring(table , encoding='utf8', method='html'))[0]
    table = table.applymap(lambda s: s[3:-2]) #clean things up
    table = table.set_index(0)
    return table
    

def scrape_amazon(asin,hdr):
    url = 'https://www.amazon.com/dp/' + asin
    rawtext = get_page(url,hdr)
    tree = html.fromstring(rawtext)
    manuf = tree.xpath("//*[@id='bylineInfo']")[0]
    
    table1 =  amazon_table(tree,'productDetails_techSpec_section_1')
    table2 =  amazon_table(tree,'productDetails_detailBullets_sections1')
    
    #TODO: get pricing information
    #TODO: get seller information
    
    return manuf.text, table1.loc['Item model number'][1]


def lookup_upc(brand,item):
    url = 'https://www.upcitemdb.com/upc/%s %s'%(brand,item)
    rawtext = get_page(url,hdr)
    tree = html.fromstring(rawtext)
    link_cand = tree.xpath("//*[@class='rImage']")
    return link_cand[0][0].text


asin_list = ['B00OJ72LHU','B00IJ0ALYS','B00INRZ5VG'] 

for i, asin  in enumerate(asin_list):
        hdr = [hdr1,hdr2][i%2]
        brand,item = scrape_amazon(asin,hdr)
        print(brand,item)
        upc = lookup_upc(brand,item)
        print(upc)

BLACK+DECKER BDCDMT120C
885911373067
DEWALT Brush
704660055225
PORTER-CABLE PCCK604L2
885911321549


In [5]:
upc = '885911321549'
url1 = 'https://www.walmart.com/search/?cat_id=0&query=%s'%upc
rawtext1 = get_page(url1,hdr1)

In [ ]:
def scrape_walmart(upc):

    'product-title-link line-clamp line-clamp-2'

    
scrape_walmart('885911321549')